<a href="https://colab.research.google.com/github/Varun0801/Deep_Learning-and-Machine_Learning_With_Python/blob/master/Crop_Recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
#importing the required libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

#Reading the csv file
data=pd.read_csv('cpdata.csv')
print(data.head())


   temperature   humidity        ph    rainfall label
0    20.879744  82.002744  6.502985  202.935536  rice
1    21.770462  80.319644  7.038096  226.655537  rice
2    23.004459  82.320763  7.840207  263.964248  rice
3    26.491096  80.158363  6.980401  242.864034  rice
4    20.130175  81.604873  7.628473  262.717340  rice


In [23]:
#Creating dummy variable for target i.e label
from sklearn.preprocessing import LabelEncoder
# Initialize encoder object
encoder = LabelEncoder()
# Fit-transform on data
data['label'] = encoder.fit_transform(data['label'])
#label= pd.get_dummies(data.label).iloc[: , 1:]
#data= pd.concat([data,label],axis=1)
#data.drop('label', axis=1,inplace=True)
print('The data present in one row of the dataset is')
print(data.head())
X = data.iloc[:,:-1]
y = data.iloc[: ,-1]
print(X.shape)
print(y.shape)

The data present in one row of the dataset is
   temperature   humidity        ph    rainfall  label
0    20.879744  82.002744  6.502985  202.935536     28
1    21.770462  80.319644  7.038096  226.655537     28
2    23.004459  82.320763  7.840207  263.964248     28
3    26.491096  80.158363  6.980401  242.864034     28
4    20.130175  81.604873  7.628473  262.717340     28
(3100, 4)
(3100,)


In [24]:
#Dividing the data into training and test set
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=6)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)
print(X_train.shape)
print(X_test.shape)

(2480, 4)
(620, 4)


In [25]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

#Four random models intitialised
log_clf_1 = LogisticRegression(random_state=0)
log_clf_2 = LogisticRegression(random_state=42)
decision_clf1 = DecisionTreeClassifier(criterion = 'entropy',random_state=0)
decision_clf2 = DecisionTreeClassifier(criterion = 'entropy', random_state=42)
#Creating a list of models
Model_List=[('Logistic Regression 1', log_clf_1),
('Logistic Regression 2', log_clf_2),
('Decision Tree 1', decision_clf1),
('Decision Tree 2', decision_clf2)]

#Initialising soft voting model
voting_clf_soft = VotingClassifier(estimators = Model_List,voting = 'soft')
#Fitting the data
voting_clf_soft.fit(X_train, y_train)
#Scoring the model for train
soft_voting_score= voting_clf_soft.score(X_train,y_train)
print("Soft Voting Train Accuracy: %.2f"%soft_voting_score)
#Scoring the model for test
soft_voting_score= voting_clf_soft.score(X_test,y_test)
print("Soft Voting Test Accuracy: %.2f"%soft_voting_score)

Soft Voting Train Accuracy: 1.00
Soft Voting Test Accuracy: 0.92


In [26]:
import pickle as clk
clk.dump(voting_clf_soft,open('recomm_model.pkl',mode = 'wb'))

In [27]:
X_test1 = X_test.iloc[:5,:]
print(X_test1)

          0         1         2         3
0 -1.112200 -1.979039  3.073686 -0.499952
1 -1.049492 -2.098256 -0.111116 -0.315662
2  3.542652 -0.294031  1.206690 -0.399938
3 -0.811835  0.814183 -0.314683 -0.104252
4 -1.125170 -2.169180 -0.555152 -0.979726


In [28]:
filename = '/content/recomm_model.pkl'
loaded_model = clk.load(open(filename, 'rb'))
result = loaded_model.score(X_test, y_test)
print(result)
y_pred1 = loaded_model.predict(X_test1)
print(y_pred1)

0.9193548387096774
[ 2  2  0 27 12]


In [29]:
pred1 = voting_clf_soft.predict(X_test1)
print(pred1)

[ 2  2  0 27 12]


In [30]:
pred_class = encoder.inverse_transform(pred1)
print(pred_class)

['Chickpea' 'Chickpea' 'Adzuki Beans' 'pomegranate' 'Peas']
